# Handle Cube Files

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
# AiiDA imports.
%load_ext aiida
%aiida

import urllib

import traitlets as tl

from surfaces_tools.widgets import cube  # import DisplayRenderedCubes, HandleCubeFiles

In [ ]:
cube_files = cube.HandleCubeFiles()
url = urllib.parse.urlsplit(jupyter_notebook_url)
nel_up = None
nel_dw = None
try:
    pk = int(urllib.parse.parse_qs(url.query)["pk"][0])
    wc = load_node(pk)
    if wc.caller.label == "CP2K_ORBITALS":
        UKS = wc.caller.outputs.dft_output_parameters["dft_type"] == "UKS"
        nel_up = int(wc.caller.outputs.dft_output_parameters["init_nel_spin1"])
        nel_dw = int(wc.caller.outputs.dft_output_parameters["init_nel_spin2"])
        if UKS:
            cube_files.nel_up = nel_up
            cube_files.nel_dw = nel_dw
            cube_files.uks = True
        else:
            cube_files.nel_up = nel_up
    elif wc.caller.label == "CP2K_GeoOpt":
        UKS = wc.caller.outputs.output_parameters["dft_type"] == "UKS"
        nel_up = int(wc.caller.outputs.output_parameters["init_nel_spin1"])
        nel_dw = int(wc.caller.outputs.output_parameters["init_nel_spin2"])
        if UKS:
            cube_files.nel_up = nel_up
            cube_files.nel_dw = nel_dw
            cube_files.uks = True
        else:
            cube_files.nel_up = nel_up
    cube_files.node_pk = pk
except KeyError:
    pass
display(cube_files)

In [ ]:
display_images = cube.DisplayRenderedCubes()
tl.dlink((cube_files, "remote_data_uuid"), (display_images, "remote_data_uuid"))
display(display_images)